In [53]:
!pip -q install gspread gspread-dataframe google-auth

In [ ]:
# =======================
# Config & imports (portable: Colab / GitHub / local)
# =======================
import io, threading, requests, pandas as pd, os, unicodedata
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from IPython.display import display

# ---- Secrets: Colab (userdata) o ENV ----
def _get_secret(name: str) -> str | None:
    # 1) Colab
    try:
        from google.colab import userdata as _ud  # solo existe en Colab
        val = _ud.get(name)
        if val:
            return val
    except Exception:
        pass
    # 2) ENV (GitHub/local)
    return os.environ.get(name)

USER_API   = _get_secret("USER_API")
SECRET_API = _get_secret("SECRET_API")
if not USER_API or not SECRET_API:
    raise ValueError(
        "Falta USER_API o SECRET_API. En Colab: agrega en 'Secretos'. "
        "En GitHub/local: define variables/Secrets de entorno."
    )

# =======================
# Parámetros
# =======================
BASE_URL   = "https://mutatio-api.gobravo.dev"
MAX_PAGES  = 2000          # límite de páginas a intentar
WINDOW     = 12            # tamaño de lote paralelo
MAX_WORKERS= 8             # hilos para paralelizar

ENDPOINTS = {
    "facturaciones": "/accounting/facturations/download",
    "reparadoras"  : "/accounting/repairs/download",
    "collections"  : "/accounting/collections/download",
}
TIPOS_COMISION = ["LIQUIDACION_COLOMBIA","MENSUALIDAD_COLOMBIA","INSCRIPCION_COLOMBIA"]

# =======================
# Sesión HTTP y auth
# =======================
def make_session():
    s = requests.Session()
    retry = Retry(
        total=3, connect=2, read=2, backoff_factor=1.2,
        status_forcelist=[502,503,504],
        allowed_methods={"POST"},
        raise_on_status=False
    )
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.mount("http://",  HTTPAdapter(max_retries=retry))
    return s

session = make_session()

_token_lock = threading.Lock()
_token_cache = {"token": None, "user": USER_API, "secret": SECRET_API}

def _get_new_token():
    r = session.post(f"{BASE_URL}/auth/generate-token",
                     json={"user": _token_cache["user"], "secret": _token_cache["secret"]},
                     headers={"Content-Type":"application/json","Accept":"application/json"},
                     timeout=(10,45))
    r.raise_for_status()
    tok = r.json().get("token")
    if not tok:
        raise RuntimeError("Auth OK pero no vino 'token'.")
    return tok

def get_token_cached(force=False):
    with _token_lock:
        if force or not _token_cache["token"]:
            _token_cache["token"] = _get_new_token()
        return _token_cache["token"]

# =======================
# Utilidades
# =======================
def _read_csv_bytes(content: bytes) -> pd.DataFrame:
    if not content or not content.strip():
        return pd.DataFrame()
    first = content.split(b"\n", 1)[0]
    sep = b';' if first.count(b';') >= first.count(b',') else b','
    return pd.read_csv(io.BytesIO(content), sep=sep.decode(), dtype=str)

def _post_csv(resource: str, page: int, body):
    """POST que devuelve bytes CSV; refresca token en 401."""
    url = f"{BASE_URL}{resource}"
    tok = get_token_cached()
    headers = {
        "Authorization": f"Bearer {tok}",
        "Accept": "text/csv, text/plain",
        "Content-Type": "application/json",
    }
    params = {"pageToDownload": str(page)}
    r = session.post(url, headers=headers, params=params, json=body, timeout=(12,90))
    if r.status_code == 401:
        tok = get_token_cached(force=True)
        headers["Authorization"] = f"Bearer {tok}"
        r = session.post(url, headers=headers, params=params, json=body, timeout=(12,90))
    if r.status_code >= 400:
        raise RuntimeError(f"[{resource}] Error {r.status_code} en page={page}. Resumen: {r.text[:300]}")
    return r.content

def _fetch_batch(resource: str, start_page: int, body, workers=MAX_WORKERS):
    """Descarga un lote de páginas en paralelo [start_page, start_page+WINDOW)."""
    pages = list(range(start_page, start_page + WINDOW))
    out = {}
    with ThreadPoolExecutor(max_workers=workers) as ex:
        futs = {ex.submit(_post_csv, resource, p, body): p for p in pages}
        for f in as_completed(futs):
            p = futs[f]
            try:
                out[p] = _read_csv_bytes(f.result())
            except Exception:
                out[p] = pd.DataFrame()
    return [out[p] for p in pages]  # ordenadas

def _download_all_pages_parallel(resource: str, body, max_pages=MAX_PAGES):
    """Descarga por lotes paralelos. Fallback a paginación 1-based si 0-based vacía."""
    frames = []

    # --- 0-based ---
    cur = 0
    while cur < max_pages:
        batch = _fetch_batch(resource, cur, body)
        if cur == 0 and all(df.empty for df in batch):  # nada -> probar 1-based
            break
        for df in batch:
            if df.empty:
                return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
            frames.append(df)
        cur += WINDOW

    if frames:
        return pd.concat(frames, ignore_index=True)

    # --- 1-based fallback ---
    frames = []
    cur = 1
    while cur < max_pages+1:
        batch = _fetch_batch(resource, cur, body)
        if cur == 1 and all(df.empty for df in batch):
            return pd.DataFrame()
        for df in batch:
            if df.empty:
                return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
            frames.append(df)
        cur += WINDOW

    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

def _download_all_pages_seq(resource: str, body, max_pages=MAX_PAGES):
    """Descarga SECUENCIAL (robusta para 0-based y 1-based)."""
    frames = []
    # 0-based
    for p in range(max_pages):
        try:
            content = _post_csv(resource, p, body)
            df = _read_csv_bytes(content)
        except Exception:
            df = pd.DataFrame()
        if df.empty:
            if p == 0:
                # 1-based fallback
                for p1 in range(1, max_pages+1):
                    try:
                        content1 = _post_csv(resource, p1, body)
                        df1 = _read_csv_bytes(content1)
                    except Exception:
                        df1 = pd.DataFrame()
                    if df1.empty:
                        break
                    frames.append(df1)
                break
            break
        frames.append(df)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# =======================
# Descargadores
# =======================
def descargar_facturaciones_tres_tipos():
    dfs = []
    for tipo in TIPOS_COMISION:
        body = [{
            "column": "Comision",
            "attribute": "commissionType",
            "columnType": "ENUM",
            "table": "facturation",
            "operation": "IGUAL_A",
            "value": tipo
        }]
        df = _download_all_pages_parallel(ENDPOINTS["facturaciones"], body)
        if not df.empty:
            dfs.append(df)
        print(f"[Facturaciones/{tipo}] filas: {0 if df is None else len(df)}")
    if not dfs:
        return pd.DataFrame()
    return pd.concat(dfs, ignore_index=True).drop_duplicates()

def descargar_reparadoras():
    # Reparadoras usa el secuencial para evitar el edge case de page 0 vacía
    return _download_all_pages_seq(ENDPOINTS["reparadoras"], body=[])

def _norm(s: str) -> str:
    import unicodedata as _ud
    return ''.join(c for c in _ud.normalize('NFKD', str(s)) if not _ud.combining(c)).lower().strip()

def filtrar_collections_cobrado(df: pd.DataFrame) -> pd.DataFrame:
    """Devuelve solo filas con Estado de cobro == Cobrado (case-insensitive)."""
    if df.empty:
        return df
    # detectar columna de estado de cobro
    norm_cols = {col: _norm(col) for col in df.columns}
    cand = None
    for col, n in norm_cols.items():
        if "estado" in n and ("cobro" in n or "cobranza" in n or "collection" in n or "charge" in n):
            cand = col
            break
    if cand is None:
        for col in df.columns:
            if _norm(col) == "estado de cobro":
                cand = col
                break
    if cand is None:
        print("⚠️ No encontré columna 'Estado de cobro'; devuelvo Collections sin filtrar.")
        return df
    mask = df[cand].astype(str).str.lower().str.contains("cobrado")
    return df[mask].reset_index(drop=True)

def descargar_collections_filtrado():
    df = _download_all_pages_parallel(ENDPOINTS["collections"], body=[])
    return filtrar_collections_cobrado(df)

# =======================
# Ejecutar todo
# =======================
get_token_cached(force=True)

c_facturaciones = descargar_facturaciones_tres_tipos()
reparadoras_df  = descargar_reparadoras()
collections_df  = descargar_collections_filtrado()  # solo 'Cobrado/COBRADO'

print(f"Facturaciones (3 tipos): {len(c_facturaciones)} filas | {len(c_facturaciones.columns)} cols")
print(f"Reparadoras           : {len(reparadoras_df)} filas | {len(reparadoras_df.columns)} cols")
print(f"Collections (Cobrado) : {len(collections_df)} filas | {len(collections_df.columns)} cols")

display(c_facturaciones.head(3))
display(reparadoras_df.head(3))
display(collections_df.head(3))

[Facturaciones/LIQUIDACION_COLOMBIA] filas: 78061
[Facturaciones/MENSUALIDAD_COLOMBIA] filas: 802836
[Facturaciones/INSCRIPCION_COLOMBIA] filas: 91294


In [ ]:
# ========= Imports =========
import io, os, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from IPython.display import display

# ========= Secrets (Colab o ENV) =========
def _get_secret(name: str):
    # 1) Colab
    try:
        from google.colab import userdata as _ud
        val = _ud.get(name)
        if val:
            return val
    except Exception:
        pass
    # 2) GitHub/local
    return os.environ.get(name)

USER_API   = _get_secret("USER_API")
SECRET_API = _get_secret("SECRET_API")
if not USER_API or not SECRET_API:
    raise ValueError("Falta USER_API o SECRET_API. En Colab: agrégalo en 'Secretos'. "
                     "En GitHub/local: define variables/Secrets de entorno.")

# ========= Constantes =========
BASE_URL  = "https://mutatio-api.gobravo.dev"
RESOURCE  = "/accounting/repairs/download"   # <-- reparadoras

# ========= Sesión con reintentos =========
def make_session():
    s = requests.Session()
    retry = Retry(
        total=3, connect=2, read=2, backoff_factor=1.2,
        status_forcelist=[502,503,504], allowed_methods={"POST"},
        raise_on_status=False
    )
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.mount("http://",  HTTPAdapter(max_retries=retry))
    return s

session = make_session()

# ========= Auth =========
def get_token(user: str, secret: str) -> str:
    r = session.post(f"{BASE_URL}/auth/generate-token",
                     json={"user": user, "secret": secret},
                     headers={"Content-Type":"application/json","Accept":"application/json"},
                     timeout=(10,45))
    r.raise_for_status()
    tok = r.json().get("token")
    if not tok:
        raise RuntimeError("Auth OK pero no vino 'token'.")
    return tok

# ========= Util: leer CSV con ; o , =========
def _leer_csv_flexible(resp) -> pd.DataFrame:
    resp.raise_for_status()
    txt = resp.text or ""
    if not txt.strip():
        return pd.DataFrame()
    first = txt.splitlines()[0]
    sep = ';' if first.count(';') >= first.count(',') else ','
    return pd.read_csv(io.StringIO(txt), sep=sep, dtype=str)

# ========= Descarga (una página) con auto-refresh de token =========
def descargar_pagina_repairs(user: str, secret: str, page: int, filtros=None) -> pd.DataFrame:
    if filtros is None:
        filtros = []  # este endpoint funciona sin body
    url = f"{BASE_URL}{RESOURCE}"

    def _do(tok):
        return session.post(
            url,
            headers={
                "Authorization": f"Bearer {tok}",
                "Accept": "text/csv, text/plain",
                "Content-Type": "application/json",
            },
            params={"pageToDownload": str(page)},
            json=filtros,
            timeout=(12,90)
        )

    token = get_token(user, secret)
    r = _do(token)
    if r.status_code == 401:
        token = get_token(user, secret)
        r = _do(token)

    if r.status_code >= 400:
        raise RuntimeError(f"Error {r.status_code} en page={page}. Resumen: {r.text[:300]}")

    return _leer_csv_flexible(r)

# ========= Descarga total (paginando) =========
def descargar_todo_repairs(user: str, secret: str, max_pages: int = 1000, filtros=None) -> pd.DataFrame:
    frames = []
    # 0-based
    for page in range(max_pages):
        df = descargar_pagina_repairs(user, secret, page=page, filtros=filtros)
        if df.empty:
            if page == 0:
                # fallback 1-based
                frames1 = []
                for p1 in range(1, max_pages+1):
                    df1 = descargar_pagina_repairs(user, secret, page=p1, filtros=filtros)
                    if df1.empty:
                        break
                    frames1.append(df1)
                    if p1 % 25 == 0:
                        print(f"[1-based] páginas leídas: {p1}")
                return pd.concat(frames1, ignore_index=True) if frames1 else pd.DataFrame()
            break
        frames.append(df)
        if page % 25 == 0 and page > 0:
            print(f"[0-based] páginas leídas: {page+1}")

    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# ========= Ejecutar =========
reparadoras_df = descargar_todo_repairs(USER_API, SECRET_API, max_pages=1200)

print(f"Filas totales: {len(reparadoras_df)} | Columnas: {len(reparadoras_df.columns)}")
display(reparadoras_df.head())
print(reparadoras_df.columns.tolist())

Filas totales: 53334 | Columnas: 28


,Id,Referencia,Berex Id,Nombre Completo,Numero de Documento,Correo Electronico,Status,Fecha de Inicio,Tipo,Deuda Resuelve,...,Fecha de baja,Tipo de Pricing,Fecha de Reactivación,Porcentaje Liquidacion,Comision Inicial,Comision Mensual,Fecha de creacion,Fecha de actualizacion,Creado por,Actualizado por
0,3,3008127181,2766,ISABEL CRISTINA GOMEZ CORTES,52455178,LILIGRIS200@GMAIL.COM,GRADUADO,30/10/2018,4,1.9257551E7,...,NaN,Tradicional,NaN,0.15,404409.0,125174.08,10/1/2023 00:00:00,17/9/2024 10:37:24,Sin registro,sgonzalez
1,4,3046559962,6487,CARLOS ALBERTO ZARTA ISAZA,14242437,VIAJERO60-@HOTMAIL.COM,BAJA_2XC,18/1/2019,5,7127230.82,...,30/4/2025,Tradicional,NaN,0.15,149672.0,35636.15,10/1/2023 00:00:00,26/5/2025 16:48:14,Sin registro,sgonzalez
2,5,3133078002,6582,LIGIA MATEUS BENAVIDES,39763454,MATEUSLIGIA848@GMAIL.COM,ESTRUCTURADO_INCUMPLIDO,22/1/2019,5,1.3117212E7,...,31/10/2023,Tradicional,30/6/2020,0.15,275461.0,65586.06,10/1/2023 00:00:00,17/9/2024 10:37:24,Sin registro,sgonzalez
3,6,3023738871,6624,LIZ ANYELI SILVA ROMERO,52272068,ANGIESILVA.SPORT@HOTMAIL.COM,ESTRUCTURADO_INCUMPLIDO,23/1/2019,5,3.8208223E7,...,30/9/2023,Tradicional,NaN,0.15,802373.0,191041.12,10/1/2023 00:00:00,17/9/2024 10:37:24,Sin registro,sgonzalez
4,7,3167671311,6703,RICARDO NAVARRO VASQUEZ,79848858,USS_ENTERPRISE_CO@YAHOO.COM,GRADUADO,25/1/2019,5,1.1942724E7,...,NaN,Tradicional,NaN,0.15,250797.0,59713.62,10/1/2023 00:00:00,17/9/2024 10:37:24,Sin registro,sgonzalez


['Id', 'Referencia', 'Berex Id', 'Nombre Completo', 'Numero de Documento', 'Correo Electronico', 'Status', 'Fecha de Inicio', 'Tipo', 'Deuda Resuelve', 'Movimientos Mensual', 'Numero de Deudas', 'Termino de programa', 'Vehiculo de ahorro', 'Empresa', 'ARD', 'Fecha de graduación', 'Deuda inicial Fija', 'Fecha de baja', 'Tipo de Pricing', 'Fecha de Reactivación', 'Porcentaje Liquidacion', 'Comision Inicial', 'Comision Mensual', 'Fecha de creacion', 'Fecha de actualizacion', 'Creado por', 'Actualizado por']


In [ ]:
import unicodedata
import pandas as pd
from IPython.display import display

def _norm(s: str) -> str:
    s = '' if s is None else str(s)
    # normaliza acentos y espacios
    s = unicodedata.normalize("NFKD", s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    return ' '.join(s.lower().strip().split())

def subset_by_names(df: pd.DataFrame, wanted: list[str]) -> pd.DataFrame:
    """Selecciona columnas con match insensible a acentos/mayúsculas y en el orden pedido."""
    if df is None or df.empty:
        return pd.DataFrame()
    # mapa normalizado -> nombre real
    norm_to_real = {}
    for c in df.columns:
        nc = _norm(c)
        # si hay duplicados normalizados, conservamos el primero
        if nc not in norm_to_real:
            norm_to_real[nc] = c
    out_cols = []
    missing  = []
    for w in wanted:
        nc = _norm(w)
        if nc in norm_to_real:
            out_cols.append(norm_to_real[nc])
        else:
            missing.append(w)
    if missing:
        print("⚠️ No encontré en el DataFrame:", missing)
    if not out_cols:
        return pd.DataFrame()
    return df.loc[:, out_cols]

# ------------------------------
# 1) reparadoras_df (con rename)
# ------------------------------
rep_wanted = [
    'Referencia',
    'Berex Id',
    'Status',
    'Fecha de Inicio',
    'Deuda Resuelve',
    'Movimientos Mensual',   # será renombrada a "Apartado Mensual"
    'Comision Inicial',
    'Comision Mensual',
    'Vehiculo de ahorro',
    'Fecha de graduación',
    'Deuda inicial Fija',
    'Fecha de baja',
    'Tipo de Pricing',
]

rep_sel = subset_by_names(reparadoras_df, rep_wanted).copy()

# renombrar "Movimientos Mensual" -> "Apartado Mensual" (resistente a acentos/mayúsculas)
for col in list(rep_sel.columns):
    if _norm(col) == _norm('Movimientos Mensual'):
        rep_sel.rename(columns={col: 'Apartado Mensual'}, inplace=True)
        break

print(f"reparadoras_df -> {rep_sel.shape[0]} filas x {rep_sel.shape[1]} cols")
display(rep_sel.head())

# -----------------------------------
# 2) c_facturaciones (solo columnas)
# -----------------------------------
fac_wanted = [
    'Id',
    'Credito',
    'Status facturacion',
    'Tipo de comision',
    'Referencia',
    'Monto',
    'Fecha de facturacion',
    'Fecha de cobro',
]
fac_sel = subset_by_names(c_facturaciones, fac_wanted).copy()
print(f"c_facturaciones -> {fac_sel.shape[0]} filas x {fac_sel.shape[1]} cols")
display(fac_sel.head())

# --------------------------------
# 3) collections_df (solo columnas)
# --------------------------------
col_wanted = [
    'Id',
    'Monto',
    'Estado de cobro',
    'Fecha de cobro',
    'Status facturacion',
    'Tipo de comision',
    'Referencia',

]
col_sel = subset_by_names(collections_df, col_wanted).copy()
print(f"collections_df -> {col_sel.shape[0]} filas x {col_sel.shape[1]} cols")
display(col_sel.head())

# (Opcional) reasignar a las variables originales si quieres seguir trabajando con las versiones reducidas
# reparadoras_df  = rep_sel
# c_facturaciones = fac_sel
# collections_df  = col_sel

reparadoras_df -> 53334 filas x 13 cols


/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Referencia,Berex Id,Status,Fecha de Inicio,Deuda Resuelve,Apartado Mensual,Comision Inicial,Comision Mensual,Vehiculo de ahorro,Fecha de graduación,Deuda inicial Fija,Fecha de baja,Tipo de Pricing
0,3008127181,2766,GRADUADO,30/10/2018,1.9257551E7,459054.0,404409.0,125174.08,POWWI,30/11/2023,1.9257551E7,NaN,Tradicional
1,3046559962,6487,BAJA_2XC,18/1/2019,7127230.82,160453.06,149672.0,35636.15,POWWI,NaN,7127230.82,30/4/2025,Tradicional
2,3133078002,6582,ESTRUCTURADO_INCUMPLIDO,22/1/2019,1.3117212E7,192903.38,275461.0,65586.06,POWWI,NaN,1.3117212E7,31/10/2023,Tradicional
3,3023738871,6624,ESTRUCTURADO_INCUMPLIDO,23/1/2019,3.8208223E7,1125998.54,802373.0,191041.12,POWWI,NaN,3.8208223E7,30/9/2023,Tradicional
4,3167671311,6703,GRADUADO,25/1/2019,1.1942724E7,267093.47,250797.0,59713.62,POWWI,31/3/2024,1.1942724E7,NaN,Tradicional


c_facturaciones -> 78061 filas x 8 cols


,Id,Credito,Status facturacion,Tipo de comision,Referencia,Monto,Fecha de facturacion,Fecha de cobro
0,1186301,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,3208712958,294118.0,4/10/2023,3/1/2024
1,1186303,false,COBRADO,LIQUIDACION_COLOMBIA,3182588339,100000.0,3/10/2023,24/10/2023
2,1186306,false,COBRO_PARCIAL_COBRADO,LIQUIDACION_COLOMBIA,3505172416,111566.0,3/10/2023,5/12/2023
3,1186311,true,COBRADO,LIQUIDACION_COLOMBIA,3233270136,853382.5499999999,4/10/2023,4/10/2023
4,1186314,false,COBRADO,LIQUIDACION_COLOMBIA,3223109455,10162.5,4/10/2023,11/10/2023


collections_df -> 708901 filas x 7 cols


,Id,Monto,Estado de cobro,Fecha de cobro,Status facturacion,Tipo de comision,Referencia
0,1,11890.0,COBRADO,15/8/2023,COBRO_PARCIAL_COBRADO,INTERESES,6007350
1,2,23136.0,COBRADO,15/8/2023,COBRO_PARCIAL_COBRADO,INTERESES,6006933
2,3,12191.0,COBRADO,15/8/2023,COBRO_PARCIAL_COBRADO,INTERESES,6003875
3,4,11671.0,COBRADO,15/8/2023,COBRO_PARCIAL_COBRADO,INTERESES,6002993R
4,5,31133.0,COBRADO,15/8/2023,COBRO_PARCIAL_COBRADO,INTERESES,6006986


In [ ]:
def filter_eq(df: pd.DataFrame, col_like: str, value: str = "COBRADO") -> pd.DataFrame:
    if df is None or df.empty:
        return pd.DataFrame(columns=[])
    target = None
    for c in df.columns:
        if _norm(c) == _norm(col_like):
            target = c
            break
    if target is None:
        print(f"⚠️ No encontré la columna '{col_like}'.")
        return pd.DataFrame(columns=df.columns)
    mask = df[target].astype(str).str.upper().str.strip().eq(value.upper())
    return df.loc[mask].reset_index(drop=True)

# Filtrar según lo que pediste:
fac_sel = filter_eq(fac_sel, "Status facturacion", "COBRADO")
col_sel = filter_eq(col_sel, "Estado de cobro", "COBRADO")

print(f"fac_sel (Status facturacion=COBRADO): {fac_sel.shape}")
print(f"col_sel (Estado de cobro=COBRADO): {col_sel.shape}")
display(fac_sel.head())
display(col_sel.head())

fac_sel (Status facturacion=COBRADO): (64259, 8)
col_sel (Estado de cobro=COBRADO): (708901, 7)


,Id,Credito,Status facturacion,Tipo de comision,Referencia,Monto,Fecha de facturacion,Fecha de cobro
0,1186303,false,COBRADO,LIQUIDACION_COLOMBIA,3182588339,100000.0,3/10/2023,24/10/2023
1,1186311,true,COBRADO,LIQUIDACION_COLOMBIA,3233270136,853382.5499999999,4/10/2023,4/10/2023
2,1186314,false,COBRADO,LIQUIDACION_COLOMBIA,3223109455,10162.5,4/10/2023,11/10/2023
3,1186316,false,COBRADO,LIQUIDACION_COLOMBIA,3142688054,2506.0499999999997,4/10/2023,6/10/2023
4,1186318,false,COBRADO,LIQUIDACION_COLOMBIA,3214595216,218440.0,4/10/2023,6/10/2023


,Id,Monto,Estado de cobro,Fecha de cobro,Status facturacion,Tipo de comision,Referencia
0,1,11890.0,COBRADO,15/8/2023,COBRO_PARCIAL_COBRADO,INTERESES,6007350
1,2,23136.0,COBRADO,15/8/2023,COBRO_PARCIAL_COBRADO,INTERESES,6006933
2,3,12191.0,COBRADO,15/8/2023,COBRO_PARCIAL_COBRADO,INTERESES,6003875
3,4,11671.0,COBRADO,15/8/2023,COBRO_PARCIAL_COBRADO,INTERESES,6002993R
4,5,31133.0,COBRADO,15/8/2023,COBRO_PARCIAL_COBRADO,INTERESES,6006986


In [ ]:
import pandas as pd

# Supongamos que ya tienes tu DataFrame col_sel cargado
# Filtrar las filas
col_sel_filtrado = col_sel[col_sel['Tipo de comision'].isin([
    'LIQUIDACION_COLOMBIA',
    'MENSUALIDAD_COLOMBIA',
    'INSCRIPCION_COLOMBIA'
])]

# Si quieres reemplazar el DataFrame original
col_sel = col_sel_filtrado

In [ ]:
import pandas as pd
import unicodedata

def _norm(s: str) -> str:
    s = '' if s is None else str(s)
    s = unicodedata.normalize("NFKD", s)
    s = ''.join(c for c in s if not unicodedata.combining(c))
    return ' '.join(s.lower().strip().split())

def align_to_template(df: pd.DataFrame, template_cols: list[str], defaults: dict | None = None) -> pd.DataFrame:
    """Devuelve un DF con EXACTAMENTE las columnas de template_cols.
       Si falta alguna, la crea con defaults[col] o NaN."""
    defaults = defaults or {}
    norm_map = {_norm(c): c for c in df.columns}
    cols = {}
    for t in template_cols:
        nt = _norm(t)
        if nt in norm_map:
            cols[t] = df[norm_map[nt]]
        else:
            cols[t] = pd.Series([defaults.get(t, pd.NA)] * len(df), index=df.index, dtype="object")
    return pd.DataFrame(cols, index=df.index)

# 1) Plantilla = columnas de fac_sel (en tu screenshot son 8 columnas)
template = fac_sel.columns.tolist()

# 2) Valor por defecto para 'Credito' en col_sel (respetando el tipo de fac_sel)
cred_default = 'false' if str(fac_sel['Credito'].dtype) == 'object' else False
defaults_col_sel = {'Credito': cred_default}

# 3) Alinear col_sel a la plantilla (esto también crea 'Fecha de facturacion' si no existe)
col_sel_aligned = align_to_template(col_sel, template, defaults=defaults_col_sel)

# 4) Concatenar
unificado = pd.concat([fac_sel, col_sel_aligned], ignore_index=True)

print(f"fac_sel: {fac_sel.shape}  |  col_sel alineado: {col_sel_aligned.shape}")
print(f"unificado: {unificado.shape}")
display(unificado.head(10))

fac_sel: (64259, 8)  |  col_sel alineado: (413910, 8)
unificado: (478169, 8)


,Id,Credito,Status facturacion,Tipo de comision,Referencia,Monto,Fecha de facturacion,Fecha de cobro
0,1186303,false,COBRADO,LIQUIDACION_COLOMBIA,3182588339,100000.0,3/10/2023,24/10/2023
1,1186311,true,COBRADO,LIQUIDACION_COLOMBIA,3233270136,853382.5499999999,4/10/2023,4/10/2023
2,1186314,false,COBRADO,LIQUIDACION_COLOMBIA,3223109455,10162.5,4/10/2023,11/10/2023
3,1186316,false,COBRADO,LIQUIDACION_COLOMBIA,3142688054,2506.0499999999997,4/10/2023,6/10/2023
4,1186318,false,COBRADO,LIQUIDACION_COLOMBIA,3214595216,218440.0,4/10/2023,6/10/2023
5,1186321,false,COBRADO,LIQUIDACION_COLOMBIA,3115705677,290452.5,4/10/2023,6/10/2023
6,1186322,false,COBRADO,LIQUIDACION_COLOMBIA,3223109455,38951.1,4/10/2023,11/10/2023
7,1186323,false,COBRADO,LIQUIDACION_COLOMBIA,3137009218,4804.8,4/10/2023,6/10/2023
8,1186324,false,COBRADO,LIQUIDACION_COLOMBIA,3168124099,49280.0,4/10/2023,10/10/2023
9,1186326,false,COBRADO,LIQUIDACION_COLOMBIA,3003971213,570380.0,4/10/2023,6/10/2023


In [ ]:
import pandas as pd
import re

def _to_number_safe(x):
    s = str(x).strip()
    if not s:
        return pd.NA

    # Limpia moneda/espacios y convierte (1.234,56) -> -1.234,56
    s = s.replace("$", "").replace(" ", "")
    s = re.sub(r'^\((.*)\)$', r'-\1', s)

    # Encuentra el último separador
    last_dot = s.rfind('.')
    last_com = s.rfind(',')

    if last_dot == -1 and last_com == -1:
        # Solo dígitos/signo
        return pd.to_numeric(s, errors="coerce")

    # El separador decimal es el que aparece más a la derecha
    if last_dot > last_com:
        dec = '.'
        thousands = ','
    else:
        dec = ','
        thousands = '.'

    # Elimina separador de miles
    s = s.replace(thousands, '')

    # Normaliza decimal a punto
    if dec == ',':
        s = s.replace(',', '.')

    return pd.to_numeric(s, errors="coerce")


# ----- 1) Partimos del DF unificado (fac_sel + col_sel alineado) -----
df = unificado.copy()

for c in ["Id", "Fecha de facturacion"]:
    if c in df.columns:
        df.drop(columns=c, inplace=True)

df["Credito"] = df["Credito"].astype(str).str.lower().str.strip()

df["Monto_num"] = df["Monto"].map(_to_number_safe).fillna(0)

df["Fecha de cobro"] = pd.to_datetime(df["Fecha de cobro"], dayfirst=True, errors="coerce")
df["Fecha de cobro"] = df["Fecha de cobro"].dt.to_period("M").dt.to_timestamp("M")

keys = ["Referencia", "Credito", "Status facturacion", "Tipo de comision", "Fecha de cobro"]
resultado = (
    df.groupby(keys, dropna=False, as_index=False)["Monto_num"]
      .sum()
      .rename(columns={"Monto_num": "Monto"})
    .sort_values(["Referencia", "Fecha de cobro"])
    .reset_index(drop=True)
)

# Orden opcional
resultado = resultado.sort_values(["Referencia", "Fecha de cobro"]).reset_index(drop=True)

print(f"Resultado: {resultado.shape}")
display(resultado.head(10))

Resultado: (301981, 6)


,Referencia,Credito,Status facturacion,Tipo de comision,Fecha de cobro,Monto
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-06-30,602031.000000
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-07-31,627346.000000
2,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-08-31,3852.000000
3,1000046281,false,COBRO_PARCIAL_INCOBRABLE,INSCRIPCION_COLOMBIA,2024-05-31,392457.000000
4,1000120635,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2024-02-29,251033.000000
5,1000120635,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2024-04-30,249418.016954
6,1000120635,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2024-05-31,3485.000000
7,1000120635,false,COBRO_PARCIAL_COBRADO,MENSUALIDAD_COLOMBIA,2024-06-30,70273.000000
8,1000120635,true,COBRADO,LIQUIDACION_COLOMBIA,2024-09-30,854192.979000
9,1000136703,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2024-07-31,276453.000000


In [ ]:
import pandas as pd

# Asegúrate de que la clave de unión tenga el mismo tipo en ambos DataFrames
resultado['Referencia'] = resultado['Referencia'].astype(str)
rep_sel['Referencia']   = rep_sel['Referencia'].astype(str)

# Hacemos un merge left: se quedan las filas de resultado y se añade la columna 'Fecha de Inicio'
resultado = resultado.merge(
    rep_sel[['Referencia', 'Fecha de Inicio']],
    on='Referencia',
    how='left'
)

In [ ]:
import pandas as pd
import numpy as np

# =========================
# 0) Normalización y utilidades
# =========================
def _norm_ref_base(s: pd.Series) -> pd.Series:
    """Referencia base: quita espacios, '.0' final y sufijos '-n'."""
    s = s.astype(str).str.strip()
    s = s.str.replace(r'\.0$', '', regex=True)    # 12345.0 -> 12345
    s = s.str.replace(r'-\d+$', '', regex=True)   # quita -1, -2, ...
    return s

def _is_empty(x: pd.Series) -> pd.Series:
    return x.isna() | (x.astype(str).str.strip() == '')

# =========================
# 1) Copias y claves base
# =========================
res = resultado.copy()
src = rep_sel.copy()

# Normalizar clave
res['__ref_base__'] = _norm_ref_base(res['Referencia'])
src['__ref_base__'] = _norm_ref_base(src['Referencia'])

# Fechas
res['Fecha de cobro']  = pd.to_datetime(res['Fecha de cobro'],  dayfirst=True, errors='coerce')
res['Fecha de Inicio'] = pd.to_datetime(res['Fecha de Inicio'], dayfirst=True, errors='coerce')
src['Fecha de Inicio'] = pd.to_datetime(src['Fecha de Inicio'], dayfirst=True, errors='coerce')

# =========================
# 2) Columnas a rellenar (mismo nombre en rep_sel y resultado)
# =========================
cols_objetivo = [
    'Fecha de Inicio',
    'Apartado Mensual',
    'Comision Inicial',
    'Comision Mensual',
    'Vehiculo de ahorro',
    'Deuda inicial Fija'
]

# Asegurar que existan en 'res'
for c in cols_objetivo:
    if c not in res.columns:
        res[c] = pd.NA

# =========================
# 3) Separar referencias únicas vs. repetidas en rep_sel
# =========================
dup_mask = src.duplicated('__ref_base__', keep=False)
src_unique = src.loc[~dup_mask].set_index('__ref_base__')
src_dups   = src.loc[dup_mask].copy()

# =========================
# 4) Relleno para referencias ÚNICAS (merge directo por map)
# =========================
for dcol in cols_objetivo:
    if dcol not in src_unique.columns:
        continue
    was_empty = _is_empty(res[dcol])
    res.loc[was_empty, dcol] = res.loc[was_empty, '__ref_base__'].map(src_unique[dcol])

# =========================
# 5) Reglas para referencias REPETIDAS
#    - "old": fila con Fecha de Inicio más antigua
#    - "new": fila con Fecha de Inicio más reciente
#    - En resultado:
#        si Fecha de cobro < FechaInicio_new → usar valores "old"
#        si Fecha de cobro ≥ FechaInicio_new O es NaT → usar valores "new"
#    - Solo rellenamos donde esté vacío
# =========================
if not src_dups.empty:
    # ordenar por fecha
    src_dups = src_dups.sort_values(['__ref_base__', 'Fecha de Inicio'])
    rules = []

    for base, g in src_dups.groupby('__ref_base__', sort=False):
        g = g.dropna(subset=['Fecha de Inicio'])
        if g.empty:
            continue
        row_old = g.iloc[0]   # más antigua
        row_new = g.iloc[-1]  # más reciente
        item = {
            '__ref_base__': base,
            'threshold': row_new['Fecha de Inicio'],  # fecha que separa old/new
        }
        for dcol in cols_objetivo:
            item[f'old:{dcol}'] = row_old.get(dcol, pd.NA)
            item[f'new:{dcol}'] = row_new.get(dcol, pd.NA)
        rules.append(item)

    if rules:
        rules_df = pd.DataFrame(rules).set_index('__ref_base__')

        # Aplicar por referencia
        groups = res.groupby('__ref_base__').groups
        for base, idx_list in groups.items():
            if base not in rules_df.index:
                continue  # esta ref no es repetida en rep_sel
            thr = rules_df.at[base, 'threshold']

            idx = pd.Index(idx_list)
            rows = res.loc[idx]

            # Máscaras por Fecha de cobro
            mask_new = (rows['Fecha de cobro'].notna() & (rows['Fecha de cobro'] >= thr)) | rows['Fecha de cobro'].isna()
            mask_old = rows['Fecha de cobro'].notna() & (rows['Fecha de cobro'] <  thr)

            # Rellenar SOLO vacíos con valores new/old
            for dcol in cols_objetivo:
                # NEW
                was_empty_new = _is_empty(rows.loc[mask_new, dcol])
                if was_empty_new.any():
                    res.loc[idx[mask_new][was_empty_new], dcol] = rules_df.at[base, f'new:{dcol}']
                # OLD
                was_empty_old = _is_empty(rows.loc[mask_old, dcol])
                if was_empty_old.any():
                    res.loc[idx[mask_old][was_empty_old], dcol] = rules_df.at[base, f'old:{dcol}']

# =========================
# 6) Salida final
# =========================
resultado_actualizado = res.drop(columns='__ref_base__').reset_index(drop=True)

# (Opcional) verificar que no cambió el número de filas ni el orden principal
assert len(resultado_actualizado) == len(resultado), "El proceso cambió el número de filas."

print("Relleno completado. Filas:", len(resultado_actualizado))

Relleno completado. Filas: 301981


In [ ]:
# Total de referencias únicas en todo el DataFrame
total_referencias = resultado['Referencia'].nunique()

# Filtrar filas con Fecha de Inicio nula
sin_fecha = resultado[resultado['Fecha de Inicio'].isna()]

# Referencias únicas sin Fecha de Inicio
referencias_sin_fecha = sin_fecha['Referencia'].unique()
cantidad_sin_fecha = len(referencias_sin_fecha)

print("Total de referencias únicas en resultado:", total_referencias)
print("Cantidad de referencias únicas sin Fecha de Inicio:", cantidad_sin_fecha)
print("Listado de referencias sin Fecha de Inicio:")
print(referencias_sin_fecha)

Total de referencias únicas en resultado: 85264
Cantidad de referencias únicas sin Fecha de Inicio: 32881
Listado de referencias sin Fecha de Inicio:
['100017' '100076' '100086' ... '99944' '99961' '99990']


In [ ]:
import os, json, pandas as pd, gspread

# ========= 1) Obtener el secreto =========
def _get_secret(name: str):
    # Intenta primero en Colab
    try:
        from google.colab import userdata as _ud
        val = _ud.get(name)
        if val:
            return val
    except Exception:
        pass
    # Luego en entorno normal (GitHub / local)
    return os.environ.get(name)

creds_json = _get_secret("MI_JSON")
if not creds_json:
    raise RuntimeError(
        "No encontré el secreto 'MI_JSON'. "
        "En Colab, agrégalo en 'Secretos'. "
        "En GitHub/local, define la variable de entorno MI_JSON con el JSON del Service Account."
    )

creds_dict = json.loads(creds_json)

# ========= 2) Autorizar gspread =========
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]
gc = gspread.service_account_from_dict(creds_dict, scopes=scopes)

# ========= 3) Abrir la hoja por ID y gid =========
sheet_id = "1JB8m0xVAJYhP2e57DcIe-MujFEXIbyRO2x76C30-9Pk"
gid      = 655613670  # pestaña específica

ws = gc.open_by_key(sheet_id).get_worksheet_by_id(gid)

# ========= 4) Traer todos los valores y recortar a primeras 23 columnas =========
valores = ws.get_all_values()
df = pd.DataFrame(valores).iloc[:, :23]   # solo columnas 0–22

# ========= 5) Mostrar muestra y shape =========
print(df.head())
print(f"Filas: {df.shape[0]}  |  Columnas: {df.shape[1]}")

         0                1                                2   \
0  ID Berex  # de Referencia               Correo electronico   
1         0       3192497430              dxlmusic0@gmail.com   
2      2290       3137159610    mauriciohenao1979@hotmail.com   
3      2291       3218454026  cielorodriguezlopez@hotmail.com   
4      2296       3163891754          wilfer.pulido@gmail.com   

                  3           4           5             6                7   \
0  Tipo de Documento      Cédula     Celular  Deuda Cierre  SUMA(N3:N17932)   
1                 CC  1090394004  3192497430     6.620.900        6.620.900   
2                 CC    71793554  3137159610     7.272.904        7.272.904   
3                 CC    40936413  3218454026    32.837.439       32.837.439   
4                 CC  1073507449  3163891754     9.805.568        9.805.568   

         8                9   ...       13        14                15  \
0  Apartado  Fecha de Inicio  ...  C Powwi  Vehículo        

In [ ]:
# Usar la primera fila como encabezados y eliminarla del cuerpo
df.columns = df.iloc[0]           # asigna la fila 0 como nombres de columnas
df = df.drop(index=0).reset_index(drop=True)  # quita la fila 0 y rein

In [ ]:
import pandas as pd
import numpy as np

# =========================
# 0) Normalización y utilidades
# =========================
def _norm_ref_base(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()
    s = s.str.replace(r'\.0$', '', regex=True)    # 12345.0 -> 12345
    s = s.str.replace(r'-\d+$', '', regex=True)   # quita -1, -2, ...
    return s

def _is_empty(x: pd.Series) -> pd.Series:
    return x.isna() | (x.astype(str).str.strip() == '')

# =========================
# 1) Copias y claves base
# =========================
res = resultado_actualizado.copy()
src = df.copy()

res['__ref_base__'] = _norm_ref_base(res['Referencia'])
src['__ref_base__'] = _norm_ref_base(src['# de Referencia'])

# Asegurar fechas
res['Fecha de cobro']  = pd.to_datetime(res['Fecha de cobro'],  dayfirst=True, errors='coerce')
res['Fecha de Inicio'] = pd.to_datetime(res['Fecha de Inicio'], dayfirst=True, errors='coerce')
src['Fecha de Inicio'] = pd.to_datetime(src['Fecha de Inicio'], dayfirst=True, errors='coerce')

# Columnas origen (en df) y destino (en resultado_actualizado)
col_mensualidad = ' Mensualidad Total' if ' Mensualidad Total' in src.columns else 'Mensualidad Total'
pairs = [
    ('Fecha de Inicio',    'Fecha de Inicio'),
    ('Apartado Mensual',   'Apartado'),
    ('Comision Mensual',   col_mensualidad),
    ('Vehiculo de ahorro', 'Vehículo'),
    ('Deuda inicial Fija', 'Deuda Cierre'),
]

# Asegurar que columnas destino existan en res
for dcol, _ in pairs:
    if dcol not in res.columns:
        res[dcol] = pd.NA

# =========================
# 2) Separar referencias únicas vs repetidas en df
# =========================
dup_mask = src.duplicated('__ref_base__', keep=False)
src_unique = src.loc[~dup_mask].set_index('__ref_base__')
src_dups   = src.loc[dup_mask].copy()

# =========================
# 3) Relleno para referencias ÚNICAS (merge directo por map)
# =========================
for dcol, scol in pairs:
    if scol not in src_unique.columns:
        continue
    was_empty = _is_empty(res[dcol])
    res.loc[was_empty, dcol] = res.loc[was_empty, '__ref_base__'].map(src_unique[scol])

# =========================
# 4) Reglas para referencias REPETIDAS:
#    - "old": fila con Fecha de Inicio más antigua
#    - "new": fila con Fecha de Inicio más reciente
# =========================
if not src_dups.empty:
    # ordenar por fecha
    src_dups = src_dups.sort_values(['__ref_base__', 'Fecha de Inicio'])
    rules = []

    for base, g in src_dups.groupby('__ref_base__', sort=False):
        g = g.dropna(subset=['Fecha de Inicio'])
        if g.empty:
            continue
        row_old = g.iloc[0]
        row_new = g.iloc[-1]
        item = {
            '__ref_base__': base,
            'threshold': row_new['Fecha de Inicio'],  # fecha que separa old/new
        }
        # guardar valores old/new para cada destino
        for dcol, scol in pairs:
            item[f'old:{dcol}'] = row_old.get(scol, pd.NA)
            item[f'new:{dcol}'] = row_new.get(scol, pd.NA)
        rules.append(item)

    if rules:
        rules_df = pd.DataFrame(rules).set_index('__ref_base__')

        # aplicar por referencia
        for base, sub_idx in res.groupby('__ref_base__').groups.items():
            if base not in rules_df.index:
                continue  # esta ref no es repetida en df
            thr = rules_df.at[base, 'threshold']

            # filas de esta ref en res
            idx = pd.Index(sub_idx)
            rows = res.loc[idx]

            # submáscaras por fecha de cobro
            mask_new = (rows['Fecha de cobro'].notna() & (rows['Fecha de cobro'] >= thr)) | rows['Fecha de cobro'].isna()
            mask_old = rows['Fecha de cobro'].notna() & (rows['Fecha de cobro'] <  thr)

            # rellenar SOLO vacíos con valores new/old
            for dcol, _ in pairs:
                # NEW
                was_empty_new = _is_empty(rows.loc[mask_new, dcol])
                if was_empty_new.any():
                    res.loc[idx[mask_new][was_empty_new], dcol] = rules_df.at[base, f'new:{dcol}']
                # OLD
                was_empty_old = _is_empty(rows.loc[mask_old, dcol])
                if was_empty_old.any():
                    res.loc[idx[mask_old][was_empty_old], dcol] = rules_df.at[base, f'old:{dcol}']

# =========================
# 5) Salida final
# =========================
resultado_actualizado = res.drop(columns='__ref_base__').reset_index(drop=True)
print("Relleno completado. Filas:", len(resultado_actualizado))

Relleno completado. Filas: 301981


In [ ]:
import pandas as pd

# 1️⃣ Filtrar filas: mantener solo las referencias que sean enteramente numéricas
#    isdigit() devuelve True solo si la cadena son dígitos del 0-9
resultado_actualizado = resultado_actualizado[resultado_actualizado['Referencia'].astype(str).str.isdigit()].copy()

# 2️⃣ Eliminar la columna 'Comision Inicial' si existe
if 'Comision Inicial' in resultado_actualizado.columns:
    resultado_actualizado.drop(columns='Comision Inicial', inplace=True)

# Verificación
print("Filas restantes:", resultado_actualizado.shape[0])
print("Columnas actuales:", list(resultado_actualizado.columns))

Filas restantes: 301922
Columnas actuales: ['Referencia', 'Credito', 'Status facturacion', 'Tipo de comision', 'Fecha de cobro', 'Monto', 'Fecha de Inicio', 'Apartado Mensual', 'Comision Mensual', 'Vehiculo de ahorro', 'Deuda inicial Fija']


In [ ]:
import pandas as pd
import numpy as np

def to_number_mixed_dot(series: pd.Series) -> pd.Series:
    # Forzamos string dtype robusto
    s = series.astype("string").str.strip()

    # Negativos con paréntesis
    s = s.str.replace(r'^\((.*)\)$', r'-\1', regex=True)

    # Notación científica -> directo
    m_sci = s.str.contains(r'^[+-]?\d*\.?\d+(e|E)[+-]?\d+$', na=False)
    out = pd.Series(np.nan, index=s.index, dtype="float64")
    if m_sci.any():
        out.loc[m_sci] = pd.to_numeric(s[m_sci], errors="coerce")

    # Resto: limpiamos todo lo que no sea dígito, punto o signo; quitamos comas (miles)
    t = s[~m_sci].copy()
    t = t.str.replace(r'[^0-9\.\-]', '', regex=True)   # quita símbolos, espacios, etc.
    t = t.str.replace(',', '', regex=False)            # (por si había)

    # Contar puntos
    dot_count = t.str.count(r'\.')

    # >1 punto => todos son miles (quitar todos los puntos)
    idx_multi = dot_count > 1
    if idx_multi.any():
        out.loc[idx_multi.index[idx_multi]] = pd.to_numeric(
            t[idx_multi].str.replace('.', '', regex=False),
            errors='coerce'
        )

    # 1 punto => decidir por nº de dígitos antes del punto
    idx_one = dot_count == 1
    if idx_one.any():
        one = t[idx_one]
        # parte antes del punto sin signo
        before = one.str.split('.', n=1).str[0].str.replace('-', '', regex=False)
        m_dec = before.str.len().astype("Int64") >= 4  # ≥4 dígitos antes => decimal
        if m_dec.any():
            out.loc[one.index[m_dec]] = pd.to_numeric(one[m_dec], errors='coerce')
        if (~m_dec).any():
            out.loc[one.index[~m_dec]] = pd.to_numeric(
                one[~m_dec].str.replace('.', '', regex=False),
                errors='coerce'
            )

    # 0 puntos => número directo
    idx_zero = dot_count == 0
    if idx_zero.any():
        out.loc[t.index[idx_zero]] = pd.to_numeric(t[idx_zero], errors='coerce')

    return out

# Aplica a tus columnas
cols_a_convertir = ['Apartado Mensual', 'Comision Mensual', 'Deuda inicial Fija']
for c in cols_a_convertir:
    if c in resultado_actualizado.columns:
        resultado_actualizado[c] = to_number_mixed_dot(resultado_actualizado[c])

# Verificación rápida
print(resultado_actualizado[cols_a_convertir].dtypes)
print(resultado_actualizado[cols_a_convertir].head(10))

/tmp/ipython-input-705226084.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m_sci = s.str.contains(r'^[+-]?\d*\.?\d+(e|E)[+-]?\d+$', na=False)
/tmp/ipython-input-705226084.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m_sci = s.str.contains(r'^[+-]?\d*\.?\d+(e|E)[+-]?\d+$', na=False)
/tmp/ipython-input-705226084.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m_sci = s.str.contains(r'^[+-]?\d*\.?\d+(e|E)[+-]?\d+$', na=False)


Apartado Mensual      float64
Comision Mensual      float64
Deuda inicial Fija    float64
dtype: object
   Apartado Mensual  Comision Mensual  Deuda inicial Fija
0         611362.66         227237.00          43346600.0
1         611362.66         227237.00          43346600.0
2         611362.66         227237.00          43346600.0
3         389316.26          67611.23          11421445.0
4         246716.02          70272.50          11953700.0
5         246716.02          70272.50          11953700.0
6         246716.02          70272.50          11953700.0
7         246716.02          70272.50          11953700.0
8         246716.02          70272.50          11953700.0
9         272569.17          71898.50          12278900.0


In [ ]:
# Total de referencias únicas en todo el DataFrame
total_referencias = resultado_actualizado['Referencia'].nunique()

# Filtrar filas con Fecha de Inicio nula
sin_fecha = resultado_actualizado[resultado_actualizado['Fecha de Inicio'].isna()]

# Referencias únicas sin Fecha de Inicio
referencias_sin_fecha = sin_fecha['Referencia'].unique()
cantidad_sin_fecha = len(referencias_sin_fecha)

print("Total de referencias únicas en resultado:", total_referencias)
print("Cantidad de referencias únicas sin Fecha de Inicio:", cantidad_sin_fecha)
print("Listado de referencias sin Fecha de Inicio:")
print(referencias_sin_fecha)

Total de referencias únicas en resultado: 85241
Cantidad de referencias únicas sin Fecha de Inicio: 7
Listado de referencias sin Fecha de Inicio:
['117857' '3013477624' '3102380535' '3142282558' '3174152116' '46827'
 '52310468']


In [ ]:
import pandas as pd
import re

# Aseguramos que sea string
ref_series = df['# de Referencia'].astype(str)

# 1️⃣ Patrón: al menos un dígito y al menos un carácter que NO sea dígito
mask = ref_series.str.contains(r'\d') & ref_series.str.contains(r'\D')

df_con_caracteres = df[mask].copy()

print(df_con_caracteres.head())
print("Total de referencias con números + otros caracteres:", df_con_caracteres.shape[0])

0   ID Berex # de Referencia            Correo electronico Tipo de Documento  \
37      2652    3015272865-1         zaydalo_5@hotmail.com                CC   
69      2816    3132042252-1  luisalvarosierra@hotmail.com                CC   
159     3779    3166055741-1    angarita.laura@hotmail.com                CC   
162     3829    3136969034-1     carlos1966.17@hotmail.com                CC   
175     3919    3134857872-1     mifloricapa2012@gmail.com                CC   

0        Cédula     Celular Deuda Cierre SUMA(N3:N17932) Apartado  \
37     52819647  3015272865    6.468.000       6.468.000   162802   
69     79467942  3132042252   16.801.350      16.801.350   390435   
159  1018428336  3208979947    8.568.042       8.568.042   238971   
162    72142244  3136969034   11.008.000      11.008.000   374272   
175    40378121  3134857872    7.581.000       7.581.000   197367   

0   Fecha de Inicio  ... C Powwi Vehículo            Status  \
37       30/10/2018  ...       0    Powwi

In [ ]:
# Total de referencias únicas en todo el DataFrame
total_referencias = resultado_actualizado['Referencia'].nunique()

# Filtrar filas con Fecha de Inicio nula
sin_fecha = resultado_actualizado[resultado_actualizado['Fecha de Inicio'].isna()]

# Referencias únicas sin Fecha de Inicio
referencias_sin_fecha = sin_fecha['Referencia'].unique()
cantidad_sin_fecha = len(referencias_sin_fecha)

print("Total de referencias únicas en resultado:", total_referencias)
print("Cantidad de referencias únicas sin Fecha de Inicio:", cantidad_sin_fecha)
print("Listado de referencias sin Fecha de Inicio:")
print(referencias_sin_fecha)

Total de referencias únicas en resultado: 85241
Cantidad de referencias únicas sin Fecha de Inicio: 7
Listado de referencias sin Fecha de Inicio:
['117857' '3013477624' '3102380535' '3142282558' '3174152116' '46827'
 '52310468']


In [ ]:
# Asegúrate de que 'Fecha de Inicio' sea de tipo datetime
resultado_actualizado['Fecha de Inicio'] = pd.to_datetime(resultado_actualizado['Fecha de Inicio'], errors='coerce')

# Crear la nueva columna 'Mes_Año' con el formato "MM-YYYY"
resultado_actualizado['Mes_Año'] = resultado_actualizado['Fecha de Inicio'].dt.strftime('%m-%Y')

In [ ]:
import numpy as np
import pandas as pd

# Asegúrate de que ambas columnas sean datetime
resultado_actualizado['Fecha de Inicio'] = pd.to_datetime(resultado_actualizado['Fecha de Inicio'], errors='coerce')
resultado_actualizado['Fecha de cobro'] = pd.to_datetime(resultado_actualizado['Fecha de cobro'], errors='coerce')

# Calcular diferencia en meses entre Fecha de Inicio y Fecha de cobro
# (año de cobro - año de inicio)*12 + (mes de cobro - mes de inicio)
resultado_actualizado['Mes_Cobro'] = (
    (resultado_actualizado['Fecha de cobro'].dt.year - resultado_actualizado['Fecha de Inicio'].dt.year) * 12 +
    (resultado_actualizado['Fecha de cobro'].dt.month - resultado_actualizado['Fecha de Inicio'].dt.month)
)

# Si Fecha de Inicio o Fecha de cobro son nulos, poner NaN
resultado_actualizado['Mes_Cobro'] = np.where(
    resultado_actualizado['Fecha de Inicio'].notna() & resultado_actualizado['Fecha de cobro'].notna(),
    resultado_actualizado['Mes_Cobro'],
    np.nan
)

In [ ]:
# Filtrar manteniendo solo los Mes_Cobro no nulos y >= 0
resultado_actualizado = resultado_actualizado[
    resultado_actualizado['Mes_Cobro'].notna() & (resultado_actualizado['Mes_Cobro'] >= 0)
].reset_index(drop=True)

In [ ]:
resultado_actualizado

,Referencia,Credito,Status facturacion,Tipo de comision,Fecha de cobro,Monto,Fecha de Inicio,Apartado Mensual,Comision Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-06-30,602031.0,2025-06-16,611362.66,227237.00,SKANDIA,43346600.0,06-2025,0.0
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-07-31,627346.0,2025-06-16,611362.66,227237.00,SKANDIA,43346600.0,06-2025,1.0
2,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-08-31,3852.0,2025-06-16,611362.66,227237.00,SKANDIA,43346600.0,06-2025,2.0
3,1000046281,false,COBRO_PARCIAL_INCOBRABLE,INSCRIPCION_COLOMBIA,2024-05-31,392457.0,2024-04-30,389316.26,67611.23,SKANDIA,11421445.0,04-2024,1.0
4,1000120635,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2024-02-29,251033.0,2024-02-10,246716.02,70272.50,SKANDIA,11953700.0,02-2024,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
300470,99990,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2021-06-30,517852.0,2021-06-16,519374.00,157577.00,A&V,29414564.0,06-2021,0.0
300471,99990,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2021-07-31,12026.0,2021-06-16,519374.00,157577.00,A&V,29414564.0,06-2021,1.0
300472,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2021-12-31,1210878.0,2021-06-16,519374.00,157577.00,A&V,29414564.0,06-2021,6.0
300473,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2022-07-31,112390.0,2021-06-16,519374.00,157577.00,A&V,29414564.0,06-2021,13.0


#Arreglo

In [ ]:
rep_sel = rep_sel[["Referencia", "Apartado Mensual", "Deuda Resuelve", "Deuda inicial Fija"]]

In [ ]:
df = df[["# de Referencia", "Apartado", "Deuda Cierre"]]

In [ ]:
# Elimina puntos y convierte a float
df["Deuda Cierre"] = (
    df["Deuda Cierre"]
      .astype(str)           # asegúrate de que sea string
      .str.replace('.', '', regex=False)  # quita separadores de miles
      .str.replace(',', '.', regex=False) # por si acaso hay comas decimales
      .astype(float)
)

In [ ]:
# 1️⃣ Primero, asegúrate de que las columnas clave tengan el mismo tipo
df['# de Referencia'] = df['# de Referencia'].astype(str)
resultado_actualizado['Referencia'] = resultado_actualizado['Referencia'].astype(str)

# 2️⃣ Crea un diccionario de referencia→Deuda Cierre para hacer el mapeo rápido
mapa_cierre = df.set_index('# de Referencia')['Deuda Cierre']

# 3️⃣ Reemplaza solo donde Deuda inicial Fija < 5,000,000
mask = resultado_actualizado['Deuda inicial Fija'] < 5_000_000
resultado_actualizado.loc[mask, 'Deuda inicial Fija'] = (
    resultado_actualizado.loc[mask, 'Referencia'].map(mapa_cierre)
)

# 4️⃣ (opcional) verifica
print(resultado_actualizado['Deuda inicial Fija'].head())

0    43346600.0
1    43346600.0
2    43346600.0
3    11421445.0
4    11953700.0
Name: Deuda inicial Fija, dtype: float64


In [ ]:
df["Deuda Cierre"] = pd.to_numeric(df["Deuda Cierre"], errors="coerce")
resultado_actualizado["Deuda inicial Fija"] = pd.to_numeric(
    resultado_actualizado["Deuda inicial Fija"], errors="coerce"
)

# Unimos por referencia
merged = resultado_actualizado.merge(
    df[["# de Referencia", "Deuda Cierre"]],
    left_on="Referencia",
    right_on="# de Referencia",
    how="left",
    suffixes=("", "_df")
)

# Creamos máscara donde la diferencia sea mayor a 50,000
mask = (merged["Deuda Cierre"].notna() &
        (merged["Deuda Cierre"] - merged["Deuda inicial Fija"]).abs() > 50000)

# Reemplazamos en resultado_actualizado solo para esas referencias
resultado_actualizado.loc[mask, "Deuda inicial Fija"] = merged.loc[mask, "Deuda Cierre"]

In [ ]:
columnas_eliminar = ['Comision Mensual']

resultado_actualizado = resultado_actualizado.drop(columns=columnas_eliminar)

In [ ]:
resultado_actualizado

,Referencia,Credito,Status facturacion,Tipo de comision,Fecha de cobro,Monto,Fecha de Inicio,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-06-30,602031.0,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,0.0
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-07-31,627346.0,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,1.0
2,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2025-08-31,3852.0,2025-06-16,611362.66,SKANDIA,43346600.0,06-2025,2.0
3,1000046281,false,COBRO_PARCIAL_INCOBRABLE,INSCRIPCION_COLOMBIA,2024-05-31,392457.0,2024-04-30,389316.26,SKANDIA,11421445.0,04-2024,1.0
4,1000120635,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2024-02-29,251033.0,2024-02-10,246716.02,SKANDIA,11953700.0,02-2024,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
300470,99990,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2021-06-30,517852.0,2021-06-16,519374.00,A&V,29414564.0,06-2021,0.0
300471,99990,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,2021-07-31,12026.0,2021-06-16,519374.00,A&V,29414564.0,06-2021,1.0
300472,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2021-12-31,1210878.0,2021-06-16,519374.00,A&V,29414564.0,06-2021,6.0
300473,99990,true,COBRADO,LIQUIDACION_COLOMBIA,2022-07-31,112390.0,2021-06-16,519374.00,A&V,29414564.0,06-2021,13.0


#Dealer

In [ ]:
import os, json, base64, pandas as pd, gspread
from google.oauth2.service_account import Credentials

# ========= 1) Cargar secreto MI_JSON (Colab o entorno normal) =========
def _get_secret(name: str):
    try:
        from google.colab import userdata as _ud
        val = _ud.get(name)
        if val:
            return val
    except Exception:
        pass
    return os.environ.get(name)

raw_json = _get_secret("MI_JSON")
if not raw_json:
    raise RuntimeError(
        "No se encontró el secreto MI_JSON. "
        "En Colab, agrégalo en 'Secretos'. "
        "En GitHub/local, define la variable de entorno MI_JSON con el JSON del Service Account."
    )

# Puede venir en texto plano o Base64
try:
    sa_info = json.loads(raw_json)
except json.JSONDecodeError:
    sa_info = json.loads(base64.b64decode(raw_json).decode("utf-8"))

# ========= 2) Autorizar gspread =========
creds = Credentials.from_service_account_info(
    sa_info,
    scopes=[
        "https://www.googleapis.com/auth/spreadsheets.readonly",
        "https://www.googleapis.com/auth/drive.readonly",
    ],
)
client = gspread.authorize(creds)

# ========= 3) Abrir la hoja de cálculo y pestaña específica =========
spreadsheet_id = "1v7Bg9ss5ZTAIA5J4DJ8THmSMHs5ILrUeLgBZkE3NCMs"
worksheet_gid  = 678352849  # GID de la pestaña
ws = client.open_by_key(spreadsheet_id).get_worksheet_by_id(worksheet_gid)

# ========= 4) Convertir a DataFrame =========
toco_dealer = pd.DataFrame(ws.get_all_records())

print("Filas:", toco_dealer.shape[0], "| Columnas:", toco_dealer.shape[1])
print(toco_dealer.head())

Filas: 116340 | Columnas: 7
  bank_reference     cm      am       dbt       vh status fecha_de_inicio
0     1140875425  79947  233090  11335500  skandia   drop      2025-04-15
1                 38641  176268   6494250            drop      2019-07-31
2     3022623789  70487  218395   9745600    powwi   drop      2025-02-21
3              1  58402  287790   9815400            drop      2019-07-26
4              1  35317  173981   5935600            drop      2019-07-19


In [ ]:
# Seleccionar y renombrar columnas
toco_dealer = toco_dealer[['am', 'dbt', 'bank_reference']].rename(
    columns={
        'am': 'Apartado Mensual',
        'dbt': 'Deuda inicial Fija',
        'bank_reference': 'Referencia'
    }
)

# Verificar resultado
print(toco_dealer.shape)

(116340, 3)


In [ ]:
# Apartado Mensual
toco_dealer['Apartado Mensual'] = (
    toco_dealer['Apartado Mensual']
    .astype(str)
    .str.replace(r'[^\d.]', '', regex=True)
    .replace('', pd.NA)                   # Reemplaza cadenas vacías por NA
    .pipe(pd.to_numeric, errors='coerce') # Convierte a float y deja NaN donde no pueda
)

# Deuda inicial Fija
toco_dealer['Deuda inicial Fija'] = (
    toco_dealer['Deuda inicial Fija']
    .astype(str)
    .str.replace(r'[^\d.]', '', regex=True)
    .replace('', pd.NA)
    .pipe(pd.to_numeric, errors='coerce')
)

# Comprobar tipos y algunos valores
print(toco_dealer.dtypes)

Apartado Mensual      float64
Deuda inicial Fija    float64
Referencia             object
dtype: object


In [ ]:
# Asegurarnos de que las columnas sean numéricas
resultado_actualizado['Apartado Mensual']   = pd.to_numeric(resultado_actualizado['Apartado Mensual'], errors='coerce')
resultado_actualizado['Deuda inicial Fija'] = pd.to_numeric(resultado_actualizado['Deuda inicial Fija'], errors='coerce')

# Calcular AM/DB (con protección ante divisiones por 0 o NaN)
resultado_actualizado['AM/DB'] = (
    resultado_actualizado['Apartado Mensual'] /
    resultado_actualizado['Deuda inicial Fija']
)

In [ ]:
resultado_actualizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300475 entries, 0 to 300474
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Referencia          300475 non-null  object        
 1   Credito             300475 non-null  object        
 2   Status facturacion  300475 non-null  object        
 3   Tipo de comision    300475 non-null  object        
 4   Fecha de cobro      300475 non-null  datetime64[ns]
 5   Monto               300475 non-null  float64       
 6   Fecha de Inicio     300475 non-null  datetime64[ns]
 7   Apartado Mensual    300475 non-null  float64       
 8   Vehiculo de ahorro  300475 non-null  object        
 9   Deuda inicial Fija  300475 non-null  float64       
 10  Mes_Año             300475 non-null  object        
 11  Mes_Cobro           300475 non-null  float64       
 12  AM/DB               300475 non-null  float64       
dtypes: datetime64[ns](2), float64

In [ ]:
import numpy as np

# 1) Asegurar tipos para la clave
resultado_actualizado['Referencia'] = resultado_actualizado['Referencia'].astype(str)
toco_dealer['Referencia']           = toco_dealer['Referencia'].astype(str)

# 2) Máscara de filas a considerar
mask = (
    (resultado_actualizado['AM/DB'] > 0.045) |
    (resultado_actualizado['AM/DB'] < 0.0099)
)

# 3) Mapas desde toco_dealer
map_apartado = dict(zip(toco_dealer['Referencia'], toco_dealer['Apartado Mensual']))
map_deuda    = dict(zip(toco_dealer['Referencia'], toco_dealer['Deuda inicial Fija']))

# 4) Series mapeadas (pueden venir con NaN si no existe la referencia)
mapped_apartado = resultado_actualizado['Referencia'].map(map_apartado)
mapped_deuda    = resultado_actualizado['Referencia'].map(map_deuda)

# 5) Reemplazar solo si: (mask) y (hay valor mapeado no nulo)
resultado_actualizado['Apartado Mensual'] = np.where(
    mask & mapped_apartado.notna(),
    mapped_apartado,
    resultado_actualizado['Apartado Mensual']
)

resultado_actualizado['Deuda inicial Fija'] = np.where(
    mask & mapped_deuda.notna(),
    mapped_deuda,
    resultado_actualizado['Deuda inicial Fija']
)

In [ ]:
Columnas_eliminar = ['Fecha de cobro', 'Fecha de Inicio']
resultado_actualizado = resultado_actualizado.drop(columns=Columnas_eliminar)

In [ ]:
# Lista de años permitidos
anios_validos = [str(a) for a in range(2021, 2031)]

# Filtrar resultado_actualizado
resultado_actualizado = resultado_actualizado[
    resultado_actualizado['Mes_Año'].str[-4:].isin(anios_validos)
].copy()

In [ ]:
import numpy as np

# Calcular cuartiles e IQR
Q1 = resultado_actualizado['AM/DB'].quantile(0.25)
Q3 = resultado_actualizado['AM/DB'].quantile(0.75)
IQR = Q3 - Q1

# Límites para definir atípicos
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Crear la columna: 1 si es atípico, 0 si no
resultado_actualizado['AM/DB_outlier'] = np.where(
    (resultado_actualizado['AM/DB'] < lower_bound) |
    (resultado_actualizado['AM/DB'] > upper_bound),
    1,   # Es atípico
    0    # No es atípico
)


In [ ]:
import numpy as np
import pandas as pd

# Definir los límites en fracción (no en %)
bins = [ -np.inf, 0.015, 0.0199, 0.025, np.inf ]
labels = [
    "< 1.5%",
    "1.5% – 1.99%",
    "2% – 2.5%",
    "> 2.5%"
]

# Crear la columna de rango
resultado_actualizado['Rango_AM_DB'] = pd.cut(
    resultado_actualizado['AM/DB'],
    bins=bins,
    labels=labels,
    right=True,   # incluye el límite superior
    include_lowest=True
)

In [ ]:
import numpy as np
import pandas as pd

# Límites en pesos (ajusta si tu columna ya está en millones)
bins_deuda   = [-np.inf, 35_000_000, 60_000_000, 100_000_000, np.inf]
labels_deuda = [
    "< 35 M",
    "35 – 60 M",
    "60 – 100 M",
    "> 100 M"
]

# Crear la nueva columna de rango
resultado_actualizado["Rango_de_deuda"] = pd.cut(
    resultado_actualizado["Deuda inicial Fija"],
    bins=bins_deuda,
    labels=labels_deuda,
    right=True,
    include_lowest=True
)

In [ ]:
# Asegurarnos de que las columnas sean numéricas
resultado_actualizado['Apartado Mensual']   = pd.to_numeric(resultado_actualizado['Apartado Mensual'], errors='coerce')
resultado_actualizado['Deuda inicial Fija'] = pd.to_numeric(resultado_actualizado['Deuda inicial Fija'], errors='coerce')

# Calcular AM/DB (con protección ante divisiones por 0 o NaN)
resultado_actualizado['AM/DB'] = (
    resultado_actualizado['Apartado Mensual'] /
    resultado_actualizado['Deuda inicial Fija']
)

In [ ]:
resultado_actualizado

,Referencia,Credito,Status facturacion,Tipo de comision,Monto,Apartado Mensual,Vehiculo de ahorro,Deuda inicial Fija,Mes_Año,Mes_Cobro,AM/DB,AM/DB_outlier,Rango_AM_DB,Rango_de_deuda
0,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,602031.0,611362.66,SKANDIA,43346600.0,06-2025,0.0,0.014104,0,< 1.5%,35 – 60 M
1,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,627346.0,611362.66,SKANDIA,43346600.0,06-2025,1.0,0.014104,0,< 1.5%,35 – 60 M
2,1000018033,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,3852.0,611362.66,SKANDIA,43346600.0,06-2025,2.0,0.014104,0,< 1.5%,35 – 60 M
3,1000046281,false,COBRO_PARCIAL_INCOBRABLE,INSCRIPCION_COLOMBIA,392457.0,389316.26,SKANDIA,11421445.0,04-2024,1.0,0.034086,0,> 2.5%,< 35 M
4,1000120635,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,251033.0,246716.02,SKANDIA,11953700.0,02-2024,0.0,0.020639,0,2% – 2.5%,< 35 M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300470,99990,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,517852.0,519374.00,A&V,29414564.0,06-2021,0.0,0.017657,0,1.5% – 1.99%,< 35 M
300471,99990,false,COBRO_PARCIAL_COBRADO,INSCRIPCION_COLOMBIA,12026.0,519374.00,A&V,29414564.0,06-2021,1.0,0.017657,0,1.5% – 1.99%,< 35 M
300472,99990,true,COBRADO,LIQUIDACION_COLOMBIA,1210878.0,519374.00,A&V,29414564.0,06-2021,6.0,0.017657,0,1.5% – 1.99%,< 35 M
300473,99990,true,COBRADO,LIQUIDACION_COLOMBIA,112390.0,519374.00,A&V,29414564.0,06-2021,13.0,0.017657,0,1.5% – 1.99%,< 35 M


In [ ]:
import pandas as pd

df = resultado_actualizado.copy()

# 0) Quitar AM/DB si existe
for c in df.columns:
    if c.strip().lower().replace(" ", "") in {"am/db", "amdb"}:
        df = df.drop(columns=[c])
        break

# 1) Detectar nombre real de la columna Mes_Año / Mes_Ano
mes_anio_col = None
for cand in ["Mes_Año", "Mes_Ano", "Mes_AÑO", "Mes_AÑO", "Mes_Ano "]:
    if cand in df.columns:
        mes_anio_col = cand
        break
if mes_anio_col is None:
    raise KeyError("No encontré la columna 'Mes_Año' / 'Mes_Ano' en el DataFrame.")

# 2) Llaves de agrupación
keys = ["Credito", "Tipo de comision", "Vehiculo de ahorro", mes_anio_col, "Mes_Cobro"]

# 3) Verificación mínima de columnas requeridas
for k in keys + ["Referencia", "Monto"]:
    if k not in df.columns:
        raise KeyError(f"Falta la columna requerida: {k}")

# 4) Agregación final: sum(Monto) y conteo de referencias únicas
df_final = (
    df.groupby(keys, dropna=False)
      .agg(
          Monto=("Monto", "sum"),
          Referencias_unicas=("Referencia", pd.Series.nunique),
      )
      .reset_index()
)

# (Opcional) reordenar columnas
df_final = df_final[keys + ["Referencias_unicas", "Monto"]]

print("Filas originales:", len(df))
print("Filas consolidadas:", len(df_final))
df_final.head()

Filas originales: 277365
Filas consolidadas: 9415


,Credito,Tipo de comision,Vehiculo de ahorro,Mes_Año,Mes_Cobro,Referencias_unicas,Monto
0,false,INSCRIPCION_COLOMBIA,A&V,01-2021,0.0,624,262666150.0
1,false,INSCRIPCION_COLOMBIA,A&V,01-2021,1.0,998,248628319.0
2,false,INSCRIPCION_COLOMBIA,A&V,01-2021,2.0,535,3819623.0
3,false,INSCRIPCION_COLOMBIA,A&V,01-2021,3.0,63,440980.0
4,false,INSCRIPCION_COLOMBIA,A&V,01-2021,4.0,17,85820.0


In [ ]:
df_final

,Credito,Tipo de comision,Vehiculo de ahorro,Mes_Año,Mes_Cobro,Referencias_unicas,Monto
0,false,INSCRIPCION_COLOMBIA,A&V,01-2021,0.0,624,262666150.0
1,false,INSCRIPCION_COLOMBIA,A&V,01-2021,1.0,998,248628319.0
2,false,INSCRIPCION_COLOMBIA,A&V,01-2021,2.0,535,3819623.0
3,false,INSCRIPCION_COLOMBIA,A&V,01-2021,3.0,63,440980.0
4,false,INSCRIPCION_COLOMBIA,A&V,01-2021,4.0,17,85820.0
...,...,...,...,...,...,...,...
9410,true,LIQUIDACION_COLOMBIA,powwi,05-2022,8.0,3,2938486.0
9411,true,LIQUIDACION_COLOMBIA,powwi,05-2022,9.0,1,6780.0
9412,true,LIQUIDACION_COLOMBIA,powwi,05-2022,13.0,1,332040.0
9413,true,LIQUIDACION_COLOMBIA,powwi,06-2022,7.0,1,1154468.0


In [ ]:
# Columnas que queremos conservar
cols = [
    "Referencia",
    "Mes_Año",
    "Apartado Mensual",
    "Deuda inicial Fija",
    "AM/DB",
    "Rango_AM_DB",
    "Rango_de_deuda"
]

# Crear el nuevo DataFrame solo con esas columnas
subset_df = resultado_actualizado[cols].copy()

# Verificamos
print("Filas:", len(subset_df))

Filas: 277365


In [ ]:
# Elimina filas duplicadas conservando la primera aparición
subset_df = subset_df.drop_duplicates().reset_index(drop=True)

In [ ]:
# 1️⃣ Ver si hay referencias duplicadas (True/False)
hay_repetidas = subset_df['Referencia'].duplicated().any()
print("¿Hay referencias repetidas?:", hay_repetidas)

# 2️⃣ Contar cuántas referencias están repetidas
num_repetidas = subset_df['Referencia'].duplicated().sum()
print("Cantidad de referencias repetidas:", num_repetidas)

# 3️⃣ Ver exactamente cuáles se repiten (con su número de apariciones)
repetidas = (
    subset_df['Referencia']
    .value_counts()
    .loc[lambda x: x > 1]
)
print("Referencias que se repiten y cuántas veces:\n", repetidas)

¿Hay referencias repetidas?: True
Cantidad de referencias repetidas: 31
Referencias que se repiten y cuántas veces:
 Referencia
3203160911    2
3104679871    2
3102023112    2
3112212499    2
3108674063    2
48993         2
3133298211    2
3168132641    2
3502656617    2
3506542570    2
3142363069    2
3106253983    2
3162854008    2
3002088935    2
3008657733    2
3107025328    2
3213416012    2
3164955843    2
3007942353    2
3102191447    2
3007506432    2
3223906173    2
3058163159    2
3112060573    2
3168346824    2
3015016773    2
3102383215    2
3205687759    2
3004406490    2
3162698727    2
3103562923    2
Name: count, dtype: int64


In [ ]:
import pandas as pd
import re

# Copia de trabajo
tmp = subset_df.copy()

# Detectar la columna Mes_Año (acepta variantes comunes)
mes_candidates = [c for c in tmp.columns
                  if re.sub(r'[\s_]', '', c).lower() in {'mesaño','mesano'}]
if not mes_candidates:
    raise KeyError("No encontré la columna 'Mes_Año' / 'Mes_Ano'.")
MES_COL = mes_candidates[0]

# Asegurar tipos numéricos (por si vienen como texto con símbolos)
tmp['Apartado Mensual']   = pd.to_numeric(tmp['Apartado Mensual'], errors='coerce')
tmp['Deuda inicial Fija'] = pd.to_numeric(tmp['Deuda inicial Fija'], errors='coerce')

# Agrupar por Mes_Año:
# - sumar Apartado Mensual y Deuda inicial Fija
# - contar referencias únicas
resumen_mes = (
    tmp.groupby(MES_COL, dropna=False)
       .agg({
           'Apartado Mensual': 'sum',
           'Deuda inicial Fija': 'sum',
           'Referencia': pd.Series.nunique
       })
       .rename(columns={'Referencia': 'Referencias_unicas'})
       .reset_index()
)

# Orden opcional de columnas
resumen_mes = resumen_mes[[MES_COL, 'Referencias_unicas', 'Apartado Mensual', 'Deuda inicial Fija']]

resumen_mes.head()

,Mes_Año,Referencias_unicas,Apartado Mensual,Deuda inicial Fija
0,01-2021,1410,6.994351e+08,3.789150e+10
1,01-2022,810,4.567878e+08,2.793051e+10
2,01-2023,2232,1.252314e+09,7.777549e+10
3,01-2024,1614,9.156202e+08,5.694583e+10
4,01-2025,1643,9.237799e+08,5.378853e+10


In [ ]:
resumen_mes

,Mes_Año,Referencias_unicas,Apartado Mensual,Deuda inicial Fija
0,01-2021,1410,6.994351e+08,3.789150e+10
1,01-2022,810,4.567878e+08,2.793051e+10
2,01-2023,2232,1.252314e+09,7.777549e+10
3,01-2024,1614,9.156202e+08,5.694583e+10
4,01-2025,1643,9.237799e+08,5.378853e+10
5,02-2021,1398,7.107968e+08,4.003481e+10
6,02-2022,850,4.705995e+08,2.845365e+10
7,02-2023,2389,1.323119e+09,8.102017e+10
8,02-2024,1705,9.615104e+08,6.026463e+10
9,02-2025,1470,8.125032e+08,4.784261e+10


In [ ]:
subset_df

,Referencia,Mes_Año,Apartado Mensual,Deuda inicial Fija,AM/DB,Rango_AM_DB,Rango_de_deuda
0,1000018033,06-2025,611362.66,43346600.0,0.014104,< 1.5%,35 – 60 M
1,1000046281,04-2024,389316.26,11421445.0,0.034086,> 2.5%,< 35 M
2,1000120635,02-2024,246716.02,11953700.0,0.020639,2% – 2.5%,< 35 M
3,1000136703,06-2024,272569.17,12278900.0,0.022198,2% – 2.5%,< 35 M
4,1000137024,10-2024,498600.59,25457250.0,0.019586,1.5% – 1.99%,< 35 M
...,...,...,...,...,...,...,...
76885,99942,06-2021,529899.00,31496350.0,0.016824,1.5% – 1.99%,< 35 M
76886,99944,06-2021,436679.00,22096800.0,0.019762,1.5% – 1.99%,< 35 M
76887,99961,06-2021,383931.00,17062100.0,0.022502,2% – 2.5%,< 35 M
76888,99969,06-2021,611331.56,37778650.0,0.016182,1.5% – 1.99%,35 – 60 M


#Exportar datos

In [ ]:
import io, os, json, base64, pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload

# ========= 1) Cargar secreto MI_JSON (Colab o entorno normal) =========
def _get_secret(name: str):
    try:
        from google.colab import userdata as _ud
        v = _ud.get(name)
        if v:
            return v
    except Exception:
        pass
    return os.environ.get(name)

raw_json = _get_secret("MI_JSON")
if not raw_json:
    raise RuntimeError(
        "No se encontró el secreto MI_JSON. "
        "En Colab, agrégalo en 'Secretos'. "
        "En GitHub/local, define la variable de entorno MI_JSON con el JSON del Service Account."
    )

# Puede venir en texto plano o Base64
try:
    sa_info = json.loads(raw_json)
except json.JSONDecodeError:
    sa_info = json.loads(base64.b64decode(raw_json).decode("utf-8"))

SCOPES = ["https://www.googleapis.com/auth/drive"]
creds  = Credentials.from_service_account_info(sa_info, scopes=SCOPES)
drive  = build("drive", "v3", credentials=creds)

# ========= 2) Configuración de destino =========
FOLDER_ID   = "1gLSyENWMairsMCea4oI8Uh2CoXjcDK5c"      # carpeta de Google Drive
TARGET_NAME = "resultado_actualizado.xlsx"             # archivo ya creado allí

def find_file_in_folder(folder_id: str, name: str):
    q = f"'{folder_id}' in parents and name = '{name}' and trashed = false"
    res = drive.files().list(q=q, fields="files(id, name)", pageSize=10).execute()
    items = res.get("files", [])
    return items[0] if items else None

meta = find_file_in_folder(FOLDER_ID, TARGET_NAME)
if not meta:
    raise RuntimeError(
        f"No encontré '{TARGET_NAME}' en la carpeta.\n"
        f"Créalo manualmente en esa carpeta y comparte como Editor con: {sa_info.get('client_email')}"
    )

file_id = meta["id"]
print(f"📄 Actualizando: {meta['name']}  |  ID: {file_id}")

# ========= 3) Preparar DataFrames =========
#   - quita 'AM/DB_outlier' si existe, no falla si no está
resultado_sin_outlier = resultado_actualizado.drop(columns=["AM"], errors="ignore")
df2 = df_final  # sin cambios

# ========= 4) Generar Excel en memoria con dos hojas =========
buf = io.BytesIO()
with pd.ExcelWriter(buf, engine="openpyxl") as writer:
    resultado_sin_outlier.to_excel(writer, index=False, sheet_name="resultado_actualizado")
    df2.to_excel(writer, index=False, sheet_name="df_final")
buf.seek(0)

# ========= 5) Subir como nueva revisión (usa la cuota del dueño) =========
media = MediaIoBaseUpload(
    buf,
    mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
    resumable=False
)

updated = drive.files().update(
    fileId=file_id,
    media_body=media,
    fields="id, webViewLink"
).execute()

print("✅ Subido (nueva revisión).")
print("🔗 Enlace:", updated["webViewLink"])
print("Service Account:", sa_info.get("client_email"))

📄 Actualizando: resultado_actualizado.xlsx  |  ID: 1knTIYEhO4GOThiJ2jRQHc4bN1NJOVz8C
✅ Subido (nueva revisión).
🔗 Enlace: https://docs.google.com/spreadsheets/d/1knTIYEhO4GOThiJ2jRQHc4bN1NJOVz8C/edit?usp=drivesdk&ouid=115000999883344388736&rtpof=true&sd=true
Service Account: colabservice@colabaccess-468021.iam.gserviceaccount.com
